<img src="https://s8.hostingkartinok.com/uploads/images/2018/08/308b49fcfbc619d629fe4604bceb67ac.jpg" width=500, height=450>
<h3 style="text-align: center;"><b>Физтех-Школа Прикладной математики и информатики (ФПМИ) МФТИ</b></h3>

---

# Embeddings

Привет! В этом домашнем задании мы с помощью эмбеддингов решим задачу семантической классификации твитов.

Для этого мы воспользуемся предобученными эмбеддингами word2vec.

Для начала скачаем датасет для семантической классификации твитов:

In [1]:
# !gdown https://drive.google.com/uc?id=1eE1FiUkXkcbw0McId4i7qY-L8hH-_Qph&export=download
!unzip archive.zip

unzip:  cannot find or open archive.zip, archive.zip.zip or archive.zip.ZIP.


Импортируем нужные библиотеки:

In [2]:
import math
import random
import string

import numpy as np
import pandas as pd
import seaborn as sns

import regex as re
import torch
import nltk
import gensim
import gensim.downloader as api
import dill as pickle

In [3]:
random.seed(42)
np.random.seed(42)
torch.random.manual_seed(42)
torch.cuda.random.manual_seed(42)
torch.cuda.random.manual_seed_all(42)

# device = "cuda" if torch.cuda.is_available() else "cpu"
device = "mps" if torch.backends.mps.is_available() else "cpu"

In [4]:
data = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin", header=None, names=["emotion", "id", "date", "flag", "user", "text"])

Посмотрим на данные:

In [5]:
data.head()

,emotion,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [6]:
data.emotion.replace(4, 1, inplace=True)

Выведем несколько примеров твитов, чтобы понимать, с чем мы имеем дело:

In [7]:
examples = data["text"].sample(10)
print("\n".join(examples))

@chrishasboobs AHHH I HOPE YOUR OK!!! 
@misstoriblack cool , i have no tweet apps  for my razr 2
@TiannaChaos i know  just family drama. its lame.hey next time u hang out with kim n u guys like have a sleepover or whatever, ill call u
School email won't open  and I have geography stuff on there to revise! *Stupid School* :'(
upper airways problem 
Going to miss Pastor's sermon on Faith... 
on lunch....dj should come eat with me 
@piginthepoke oh why are you feeling like that? 
gahh noo!peyton needs to live!this is horrible 
@mrstessyman thank you glad you like it! There is a product review bit on the site  Enjoy knitting it!


Как вилим, тексты твитов очень "грязные". Нужно предобработать датасет, прежде чем строить для него модель классификации.

Чтобы сравнивать различные методы обработки текста/модели/прочее, разделим датасет на dev(для обучения модели) и test(для получения качества модели).

In [8]:
indexes = np.arange(data.shape[0])
np.random.shuffle(indexes)
dev_size = math.ceil(data.shape[0] * 0.8)

dev_indexes = indexes[:dev_size]
test_indexes = indexes[dev_size:]

dev_data = data.iloc[dev_indexes]
test_data = data.iloc[test_indexes]

dev_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

## Обработка текста

Токенизируем текст, избавимся от знаков пунктуации и выкинем все слова, состоящие менее чем из 4 букв:

In [9]:
tokenizer = nltk.WordPunctTokenizer()
line = tokenizer.tokenize(dev_data["text"][0].lower())
print(dev_data.text[0])
print(" ".join(line))
print(line)

@Claire_Nelson i'm on the north devon coast the next few weeks  will be down in Devon again in may sometime i hope though!
@ claire_nelson i ' m on the north devon coast the next few weeks will be down in devon again in may sometime i hope though !
['@', 'claire_nelson', 'i', "'", 'm', 'on', 'the', 'north', 'devon', 'coast', 'the', 'next', 'few', 'weeks', 'will', 'be', 'down', 'in', 'devon', 'again', 'in', 'may', 'sometime', 'i', 'hope', 'though', '!']


In [10]:
filtered_line = [w for w in line if all(c not in string.punctuation for c in w) and len(w) > 3]
print(" ".join(filtered_line))

north devon coast next weeks will down devon again sometime hope though


Загрузим предобученную модель эмбеддингов. 

Если хотите, можно попробовать другую. Полный список можно найти здесь: https://github.com/RaRe-Technologies/gensim-data.

Данная модель выдает эмбеддинги для **слов**. Строить по эмбеддингам слов эмбеддинги предложений мы будем ниже.

In [11]:
word2vec = api.load("word2vec-google-news-300")

In [12]:
emb_line = [word2vec.get_vector(w) for w in filtered_line if w in word2vec]
print(sum(emb_line).shape)

(300,)


Нормализуем эмбеддинги, прежде чем обучать на них сеть. 
(наверное, вы помните, что нейронные сети гораздо лучше обучаются на нормализованных данных)

In [13]:
mean = np.mean(word2vec.vectors, 0)
std = np.std(word2vec.vectors, 0)
norm_emb_line = [(word2vec.get_vector(w) - mean) / std for w in filtered_line if w in word2vec and len(w) > 3]
print(sum(norm_emb_line).shape)
print([all(norm_emb_line[i] == emb_line[i]) for i in range(len(emb_line))])

(300,)
[False, False, False, False, False, False, False, False, False, False, False, False]


Сделаем датасет, который будет по запросу возвращать подготовленные данные.

In [14]:
from torch.utils.data import Dataset, random_split
from nltk.corpus import stopwords

class TwitterDataset(Dataset):
    def __init__(self, data: pd.DataFrame, feature_column: str, target_column: str, word2vec: gensim.models.Word2Vec):
        self.tokenizer = nltk.WordPunctTokenizer()
        self.stop_words = set(stopwords.words('english'))
        
        self.data = data
        
        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)

    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"^@(\w+)\b", "", text)
        tokenized_text = self.tokenizer.tokenize(text.lower())
        filted_stops_and_punct_tokens = [token for token in tokenized_text
                                  if all(ch not in string.punctuation for ch in token)
                                  and token not in self.stop_words]
        return filted_stops_and_punct_tokens
        
    def get_embeddings_(self, tokens):
        # Получи эмбеддинги слов и усредни их (наверное нужно нормализовать их)
        embeddings = [(self.word2vec.get_vector(w) - self.mean) / self.std 
                      for w in tokens if w in self.word2vec and len(w) > 3]
        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [15]:
dev = TwitterDataset(dev_data, "text", "emotion", word2vec)

Отлично, мы готовы с помощью эмбеддингов слов превращать твиты в векторы и обучать нейронную сеть.

Превращать твиты в векторы, используя эмбеддинги слов, можно несколькими способами. А именно такими:

## Average embedding (2 балла)
---
Это самый простой вариант, как получить вектор предложения, используя векторные представления слов в предложении. А именно: вектор предложения есть средний вектор всех слов в предлоежнии (которые остались после токенизации и удаления коротких слов, конечно). 

In [16]:
indexes = np.arange(len(dev))
np.random.shuffle(indexes)
example_indexes = indexes[::1000]

examples = {"features": [np.sum(dev[i]["feature"], axis=0) for i in example_indexes], 
            "targets": [dev[i]["target"] for i in example_indexes]}
print(len(examples["features"]))

1280


In [17]:
unsqueeze_func = lambda x: np.expand_dims(x, axis=1)

Давайте сделаем визуализацию полученных векторов твитов тренировочного (dev) датасета. Так мы увидим, насколько хорошо твиты с разными target значениями отделяются друг от друга, т.е. насколько хорошо усреднение эмбеддингов слов предложения передает информацию о предложении.

Для визуализации векторов надо получить их проекцию на плоскость. Сделаем это с помощью `PCA`. Если хотите, можете вместо PCA использовать TSNE: так у вас получится более точная проекция на плоскость (а значит, более информативная, т.е. отражающая реальное положение векторов твитов в пространстве). Но TSNE будет работать намного дольше.

In [18]:
from sklearn.decomposition import PCA

# Обучи PCA на эмбеддингах слов
pca = PCA(n_components=2)
examples["transformed_features"] = pca.fit_transform(np.concatenate(tuple([np.array(unsqueeze_func(sent).T) 
                                                                           for sent in examples["features"]]),                                                      axis=0))

In [19]:
import bokeh.models as bm, bokeh.plotting as pl
from bokeh.io import output_notebook
output_notebook()

def draw_vectors(x, y, radius=10, alpha=0.25, color='blue',
                 width=600, height=400, show=True, **kwargs):
    """ draws an interactive plot for data points with auxilirary info on hover """
    data_source = bm.ColumnDataSource({ 'x' : x, 'y' : y, 'color': color, **kwargs })

    fig = pl.figure(active_scroll='wheel_zoom', width=width, height=height)
    fig.scatter('x', 'y', size=radius, color='color', alpha=alpha, source=data_source)

    fig.add_tools(bm.HoverTool(tooltips=[(key, "@" + key) for key in kwargs.keys()]))
    if show: pl.show(fig)
    return fig

Loading BokehJS ...

In [20]:
draw_vectors(
    examples["transformed_features"][:, 0], 
    examples["transformed_features"][:, 1], 
    color=[["red", "blue"][t] for t in examples["targets"]]
    )

Figure(id='1004', ...)

Скорее всего, на визуализации нет четкого разделения твитов между классами. Это значит, что по полученным нами векторам твитов не так-то просто определить, к какому классу твит пренадлежит. Значит, обычный линейный классификатор не очень хорошо справится с задачей. Надо будет делать глубокую (хотя бы два слоя) нейронную сеть.

Подготовим загрузчики данных.
Усреднее векторов будем делать в "батчевалке"(`collate_fn`). Она используется для того, чтобы собирать из данных `torch.Tensor` батчи, которые можно отправлять в модель.


In [21]:
from torch.utils.data import DataLoader


batch_size = 1024
num_workers = 4 if not torch.backends.mps.is_available() else 0

def average_emb(batch):
    features = np.array([np.mean(b["feature"], axis=0) for b in batch])
    targets = np.array([b["target"] for b in batch])

    return {"features": torch.FloatTensor(features), "targets": torch.LongTensor(targets)}


train_size = math.ceil(len(dev) * 0.8)

train, valid = random_split(dev, [train_size, len(dev) - train_size])

train_loader = DataLoader(train, batch_size=batch_size, num_workers=num_workers, shuffle=True, drop_last=True, collate_fn=average_emb)
valid_loader = DataLoader(valid, batch_size=batch_size, num_workers=num_workers, shuffle=False, drop_last=False, collate_fn=average_emb)

Определим функции для тренировки и теста модели:

In [22]:
from tqdm.notebook import tqdm


def training(model, optimizer, criterion, train_loader, epoch, scheduler, device="cpu"):
    pbar = tqdm(train_loader, desc=f"Epoch {epoch + 1}. Train Loss: {0}")
    model.to(device)
    model.train()
    
    
    for batch in pbar:
        features = batch["features"].to(device)
        targets = batch["targets"].to(device)

        optimizer.zero_grad()
        preds = model(features)
        # Получи предсказания модели
        loss = criterion(preds, targets)# Посчитай лосс
        loss.backward()
        # Обнови параметры модели
        optimizer.step()
        
        
        pbar.set_description(f"Epoch {e + 1}. Train Loss: {loss:.4}")
    scheduler.step()

def testing(model, criterion, test_loader, device="cpu"):
    pbar = tqdm(test_loader, desc=f"Test Loss: {0}, Test Acc: {0}")
    mean_loss = 0
    mean_acc = 0
    model.to(device)
    model.eval()
    with torch.no_grad():
        for batch in pbar:
            features = batch["features"].to(device)
            targets = batch["targets"].to(device)

            # Получи предсказания модели
            preds = model(features)
            loss = criterion(preds, targets) # Посчитай лосс
            pred_labels = torch.argmax(preds, 1)
            acc = torch.sum(pred_labels == targets) / batch_size # Посчитай точность модели

            mean_loss += loss.item()
            mean_acc += acc.item()

            pbar.set_description(f"Test Loss: {loss:.4}, Test Acc: {acc:.4}")

    pbar.set_description(f"Test Loss: {mean_loss / len(test_loader):.4}, Test Acc: {mean_acc / len(test_loader):.4}")

    return {"Test Loss": mean_loss / len(test_loader), "Test Acc": mean_acc / len(test_loader)}

Создадим модель, оптимизатор и целевую функцию. Вы можете сами выбрать количество слоев в нейронной сети, ваш любимый оптимизатор и целевую функцию.


In [23]:
import torch.nn as nn
from torch.optim import Adam, lr_scheduler
from collections import OrderedDict

# Не забудь поиграться с параметрами ;)
vector_size = dev.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 3

#модель не была построена очень глубокой, потому что скор в данном задании не так важен
#параметры обучения модели были подобраны
model = nn.Sequential(OrderedDict([
    ("linear1", nn.Linear(vector_size, 256)),
    ("activation1", nn.LeakyReLU()),
    ("batch_norm1", nn.BatchNorm1d(256)),
    ("linear2", nn.Linear(256, 128)),
    ("activation2", nn.LeakyReLU()),
    ("batch_norm2", nn.BatchNorm1d(128)),
    ("linear3", nn.Linear(128, 64)),
    ("activation3", nn.LeakyReLU()),
    ("batch_norm3", nn.BatchNorm1d(64)),
    ("linear4", nn.Linear(64, 32)),
    ("activation4", nn.LeakyReLU()),
    ("batch_norm4", nn.BatchNorm1d(32)),
    ("linear5", nn.Linear(32, 16)),
    ("activation4", nn.LeakyReLU()),
    ("linear6", nn.Linear(16, num_classes)),
]))# Твоя модель
model = model.to(device)
criterion = nn.CrossEntropyLoss()# Твой лосс
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

Наконец, обучим модель и протестируем её.

После каждой эпохи будем проверять качество модели на валидационной части датасета. Если метрика стала лучше, будем сохранять модель. **Подумайте, какая метрика (точность или лосс) будет лучше работать в этой задаче?** 

In [24]:
print(f"Распределение таргета {dev_data.emotion.sum() / dev_data.emotion.shape[0]:.3}")

Распределение таргета 0.5


In [25]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, optimizer, criterion, train_loader, e, exp_lr_scheduler, device)
    log = testing(model, criterion, valid_loader, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5154001960754394, 'Test Acc': 0.741171875}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5065038874149322, 'Test Acc': 0.74887109375}


Epoch 3. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.498893779873848, 'Test Acc': 0.75318359375}


In [26]:
test_loader = DataLoader(
    TwitterDataset(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5003211165007692, 'Test Acc': 0.7511200828674122}


**Т.к. в нашей задаче ключевой метрикой является accuracy и классы сбалансированы, то точность является более явным показателем для мониторинга данной задачи**

## Embeddings for unknown words (8 баллов)

Пока что использовалась не вся информация из текста. Часть информации фильтровалось – если слова не было в словаре эмбеддингов, то мы просто превращали слово в нулевой вектор. Хочется использовать информацию по-максимуму. Поэтому рассмотрим другие способы обработки слов, которых нет в словаре. А именно:

- Для каждого незнакомого слова будем запоминать его контекст(слова слева и справа от этого слова). Эмбеддингом нашего незнакомого слова будет сумма эмбеддингов всех слов из его контекста. (4 балла)
- Для каждого слова текста получим его эмбеддинг из Tfidf с помощью ```TfidfVectorizer``` из [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html#sklearn.feature_extraction.text.TfidfVectorizer). Итоговым эмбеддингом для каждого слова будет сумма двух эмбеддингов: предобученного и Tfidf-ного. Для слов, которых нет в словаре предобученных эмбеддингов, результирующий эмбеддинг будет просто полученный из Tfidf. (4 балла)

Реализуйте оба варианта **ниже**. Напишите, какой способ сработал лучше и ваши мысли, почему так получилось.

***Пояснение от выполняющего: в данной части будет использовано большое количество добущений, которые по моему мнению кажутся верными***

### Контекстное представление слова

In [27]:
from torch.utils.data import Dataset, random_split
from nltk.corpus import stopwords

class TwitterDatasetNeighbors(TwitterDataset):
    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"^@(\w+)\b", "", text)
        tokenized_text = self.tokenizer.tokenize(text.lower())
        filted_stops_and_punct_tokens = [token for token in tokenized_text
                                  if all(ch not in string.punctuation for ch in token) 
                                         and len(token)>2]
        return filted_stops_and_punct_tokens
       
    def get_embeddings_(self, tokens, neighbors_window=2):
        embeddings = []
        if len(tokens) == 0:
            return np.zeros((1, self.word2vec.vector_size))
        else:
            for i, tok in enumerate(tokens):
                if tok in self.word2vec:#если есть слово в словаре делаем его эмбеддинг
                    embeddings.append(self.word2vec.get_vector(tok))
                else:
                    neighbors_emb_list = []
                    start_idx = max(0, i - neighbors_window)
                    end_idx = min(len(tokens), i + neighbors_window + 1)

                    for j in range(start_idx, end_idx):
                        if i != j:
                            if tokens[j] in self.word2vec:
                                neighbors_emb_list.append(self.word2vec.get_vector(tokens[j]))  
                            else:
                                # нулевой вектор должен быть нулевым при нормировке
                                neighbors_emb_list.append(np.zeros(self.word2vec.vector_size) + self.mean)
                    if len(neighbors_emb_list) != 0:
                        embeddings.append(np.concatenate(neighbors_emb_list, axis=0).mean(axis=0))
                    else:
                        embeddings.append(np.zeros(self.word2vec.vector_size) + self.mean)

            embeddings = np.array([(emb - self.mean) / self.std for emb in embeddings])
            if len(embeddings.shape) == 1:
                    embeddings = embeddings.reshape(-1, 1)
            return embeddings

In [28]:
dev_neighbors = TwitterDatasetNeighbors(dev_data, "text", "emotion", word2vec)

In [29]:
train_size_neigh = math.ceil(len(dev_neighbors) * 0.8)

train_neigh, valid_neigh = random_split(dev_neighbors, 
                                        [train_size_neigh, len(dev_neighbors) - train_size_neigh])

train_loader_neigh = DataLoader(train_neigh, 
                                batch_size=batch_size, 
                                num_workers=num_workers, 
                                shuffle=True, 
                                drop_last=True,
                                collate_fn=average_emb)
valid_loader_neigh = DataLoader(valid_neigh, 
                                batch_size=batch_size, 
                                num_workers=num_workers, 
                                shuffle=False, 
                                drop_last=False, 
                                collate_fn=average_emb)

In [30]:
# Не забудь поиграться с параметрами ;)
vector_size = dev_neighbors.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 4

#модель не была построена очень глубокой, потому что скор в данном задании не так важен
#параметры обучения модели были подобраны
model = nn.Sequential(OrderedDict([
    ("linear1", nn.Linear(vector_size, 256)),
    ("activation1", nn.LeakyReLU()),
    ("batch_norm1", nn.BatchNorm1d(256)),
    ("linear2", nn.Linear(256, 128)),
    ("activation2", nn.LeakyReLU()),
    ("batch_norm2", nn.BatchNorm1d(128)),
    ("linear3", nn.Linear(128, 64)),
    ("activation3", nn.LeakyReLU()),
    ("batch_norm3", nn.BatchNorm1d(64)),
    ("linear4", nn.Linear(64, 32)),
    ("activation4", nn.LeakyReLU()),
    ("batch_norm4", nn.BatchNorm1d(32)),
    ("linear5", nn.Linear(32, 16)),
    ("activation4", nn.LeakyReLU()),
    ("linear6", nn.Linear(16, num_classes)),
]))# Твоя модель
model = model.to(device)
criterion = nn.CrossEntropyLoss()# Твой лосс
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.1)

In [31]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, 
             optimizer, 
             criterion, 
             train_loader_neigh, 
             e, 
             exp_lr_scheduler, 
             device)
    log = testing(model, criterion, valid_loader_neigh, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_neighbors.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.47856474459171294, 'Test Acc': 0.76765234375}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

...

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.4576653280258179, 'Test Acc': 0.77996484375}


Epoch 4. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.45614716601371763, 'Test Acc': 0.78106640625}


In [32]:
test_loader = DataLoader(
    TwitterDatasetNeighbors(test_data, "text", "emotion", word2vec), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_neighbors.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.45521795825836375, 'Test Acc': 0.7808038388578274}


**Изменение размера окна критически не полияло на качество модели (может быть связано с процессом токенизации - мы пропускаем часть токенов), но в ходе экспериментов было решено учитывать окно размера 2**

### Представление с помощью TF-IDF

**TF-IDF представление будет реализовываться сразу в коде создания датасета. Идея заключается в том, что представление каждого слова будет уменьшено до 300, чтобы можно было сложить с вектором из word2vec**

In [33]:
from torch.utils.data import Dataset, random_split
from nltk.corpus import stopwords
from sklearn import decomposition
from sklearn.feature_extraction.text import TfidfVectorizer

class TwitterDatasetTFIDF(Dataset):
    def __init__(self, data: pd.DataFrame, 
                 feature_column: str, 
                 target_column: str, 
                 word2vec: gensim.models.Word2Vec,
                 tfidf_df: pd.core.frame.DataFrame = None):
        
        
        self.tokenizer = nltk.WordPunctTokenizer()
        self.stop_words = set(stopwords.words('english'))
        
        self.data = data
        
        self.feature_column = feature_column
        self.target_column = target_column

        self.word2vec = word2vec

        self.label2num = lambda label: 0 if label == 0 else 1
        self.mean = np.mean(word2vec.vectors, axis=0)
        self.std = np.std(word2vec.vectors, axis=0)
        
        self.tfidf_df = tfidf_df
        if self.tfidf_df is None:
            self.reductor = decomposition.TruncatedSVD(n_components=300, 
                                                  random_state=69)
            self.tfidf_vect = TfidfVectorizer(min_df=5, 
                                              tokenizer=tokenizer.tokenize,
                                              token_pattern=None,
                                              stop_words="english")
            
            transformed_corpus = self.tfidf_vect.fit_transform(self.data[self.feature_column]).T
            reducted_transformed_corpus = self.reductor.fit_transform(transformed_corpus)
            
            self.tfidf_df = pd.DataFrame(reducted_transformed_corpus, 
                                         index=self.tfidf_vect.get_feature_names_out(),
                                         columns=range(self.word2vec.vector_size))
            
            del transformed_corpus, reducted_transformed_corpus
            
    def __getitem__(self, item):
        text = self.data[self.feature_column][item]
        label = self.label2num(self.data[self.target_column][item])

        tokens = self.get_tokens_(text)
        embeddings = self.get_embeddings_(tokens)

        return {"feature": embeddings, "target": label}

    def get_tokens_(self, text):
        # Получи все токены из текста и профильтруй их
        text = re.sub(r"^@(\w+)\b", "", text)
        tokenized_text = self.tokenizer.tokenize(text.lower())
        filted_stops_and_punct_tokens = [token for token in tokenized_text
                                  if all(ch not in string.punctuation for ch in token)
                                  and token not in self.stop_words and len(token)>2]
        return filted_stops_and_punct_tokens
        
    def get_embeddings_(self, tokens):
        embeddings = []
        for token in tokens:
            try:
                tfidf_emb = self.tfidf_df.loc[token]
            except KeyError:
                tfidf_emb = np.zeros(self.word2vec.vector_size)+self.mean
                
            if token in self.word2vec:
                embeddings.append((tfidf_emb + 
                                  self.word2vec.get_vector(token) -
                                  self.mean) / self.std)
            else:
                embeddings.append((tfidf_emb -
                                  self.mean) / self.std)
        # Получи эмбеддинги слов и усредни их (наверное нужно нормализовать их)

        if len(embeddings) == 0:
            embeddings = np.zeros((1, self.word2vec.vector_size))
        else:
            embeddings = np.array(embeddings)
            if len(embeddings.shape) == 1:
                embeddings = embeddings.reshape(-1, 1)

        return embeddings

    def __len__(self):
        return self.data.shape[0]

In [34]:
dev_tfidf = TwitterDatasetTFIDF(dev_data, "text", "emotion", word2vec)

In [35]:
train_size_tfidf = math.ceil(len(dev_tfidf) * 0.8)

train_tfidf, valid_tfidf = random_split(dev_tfidf, 
                                        [train_size_tfidf, len(dev_tfidf) - train_size_tfidf])

train_loader_tfidf = DataLoader(train_tfidf, 
                                batch_size=batch_size, 
                                num_workers=num_workers, 
                                shuffle=True, 
                                drop_last=True,
                                collate_fn=average_emb)
valid_loader_tfidf = DataLoader(valid_tfidf, 
                                batch_size=batch_size, 
                                num_workers=num_workers, 
                                shuffle=False, 
                                drop_last=False, 
                                collate_fn=average_emb)

In [36]:
# Не забудь поиграться с параметрами ;)
vector_size = dev_tfidf.word2vec.vector_size
num_classes = 2
lr = 1e-2
num_epochs = 2

#модель не была построена очень глубокой, потому что скор в данном задании не так важен
#параметры обучения модели были подобраны
model = nn.Sequential(OrderedDict([
    ("linear1", nn.Linear(vector_size, 256)),
    ("activation1", nn.LeakyReLU()),
    ("batch_norm1", nn.BatchNorm1d(256)),
    ("linear2", nn.Linear(256, 128)),
    ("activation2", nn.LeakyReLU()),
    ("batch_norm2", nn.BatchNorm1d(128)),
    ("linear3", nn.Linear(128, 64)),
    ("activation3", nn.LeakyReLU()),
    ("batch_norm3", nn.BatchNorm1d(64)),
    ("linear4", nn.Linear(64, 32)),
    ("activation4", nn.LeakyReLU()),
    ("batch_norm4", nn.BatchNorm1d(32)),
    ("linear5", nn.Linear(32, 16)),
    ("activation4", nn.LeakyReLU()),
    ("linear6", nn.Linear(16, num_classes)),
]))# Твоя модель
model = model.to(device)
criterion = nn.CrossEntropyLoss()# Твой лосс
optimizer = Adam(model.parameters(), lr=lr)# Твой оптимайзер
exp_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.1)

In [37]:
best_metric = np.inf
for e in range(num_epochs):
    training(model, 
             optimizer, 
             criterion, 
             train_loader_tfidf, 
             e, 
             exp_lr_scheduler, 
             device)
    log = testing(model, criterion, valid_loader_tfidf, device)
    print(log)
    if log["Test Loss"] < best_metric:
        torch.save(model.state_dict(), "model_tfidf.pt")
        best_metric = log["Test Loss"]

Epoch 1. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5167725727558136, 'Test Acc': 0.73965625}


Epoch 2. Train Loss: 0:   0%|          | 0/1000 [00:00<?, ?it/s]

Test Loss: 0, Test Acc: 0:   0%|          | 0/250 [00:00<?, ?it/s]

{'Test Loss': 0.5042776895761489, 'Test Acc': 0.74845703125}


In [39]:
test_loader = DataLoader(
    TwitterDatasetTFIDF(test_data, "text", "emotion", word2vec, dev_tfidf.tfidf_df), 
    batch_size=batch_size, 
    num_workers=num_workers, 
    shuffle=False,
    drop_last=False, 
    collate_fn=average_emb)

model.load_state_dict(torch.load("model_tfidf.pt", map_location=device))

print(testing(model, criterion, test_loader, device=device))

Test Loss: 0, Test Acc: 0:   0%|          | 0/313 [00:00<?, ?it/s]

{'Test Loss': 0.5032428600155888, 'Test Acc': 0.7490608775958466}


### Выводы:
1) В ходе этого задания была рассмотрена возможность представления предложения как среднего значения входящих в него слов, что также является оправданным подходом, как и представления в виде суммы слов.

1) Целью домашнего задания было изучения методов представления предложений, поэтому нейронная сеть классификации не была сильно глубокая и параметры моделы были подобраны на небольшом количестве обучающих циклов. Архитектура для всех моделей была одинаковая. Поэтому если постараться, можно выбить и более высокую точность.

1) Среди 3 подходов наилучшим по метрике оказался метод использования контекста для описания незнакомых слов, так же этот подход имеет небольшое время исполнение. Изменение размера окна соседей, на которых мы будет смотреть, критически не повлияло на качество модели, но наибольшая точность была достигнута при размере окна 2.

1) Хорошую точность показал алгоритм с применением TF-IDF представления. Однако у этого принципа есть свои недостатки:
- время представления TF-IDF для большого набора данных велико (TF-IDF+уменьшение размерности)
- алгоритм плохо работает с новыми словами